In [ ]:
# Google Colaboratory를 사용할 때는 다음 주석을 해제하고 실행하기 바랍니다.

In [ ]:
# # 다음을 실행하면 authorization code 입력을 요청받습니다.
# # 출력된 링크를 클릭하고 Google 계정으로 로그인한 뒤,
# # authorization code를 복사해서 붙여 넣습니다.
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# working_dir = 'MLSys_100Knocks' #　※※ 여러분이 만든 폴더 경로가 다를 때는 다음을 변경합니다. ※※
# path = f'/content/drive/MyDrive/{working_dir}/MainChapter/chapter04'
# os.chdir(path)

In [ ]:
# Jupyter notebook ipywidgets 활성화
# for jupyter notebook (virtualenv 사용 시)
#!jupyter nbextension enable --user --py widgetsnbextension

# for jupyter lab
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

# 4장 보고 구조를 만들기 위한 테크닉 10

### 테크닉 31: 특정 매장의 매출을 엑셀로 출력하자

In [1]:
import pandas as pd
import glob
import os

m_store = pd.read_csv('m_store.csv')
m_area = pd.read_csv('m_area.csv')

current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir, 'tbl_order_*.csv')
tbl_order_files = glob.glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
    order_tmp = pd.read_csv(file)
    print(f'{file}:{len(order_tmp)}')
    order_all = pd.concat([order_all, order_tmp], ignore_index=True)

# 유지 보수용 매장 데이터 삭제
order_all = order_all.loc[order_all['store_id'] != 999]

order_all = pd.merge(order_all, m_store, on='store_id', how='left')
order_all = pd.merge(order_all, m_area, on='area_cd', how='left')

# 마스터에 없는 코드에 대응한 문자열 설정
order_all.loc[order_all['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
order_all.loc[order_all['takeout_flag'] == 1, 'takeout_name'] = 'takeout'

order_all.loc[order_all['status'] == 0, 'status_name'] = '주문 접수'
order_all.loc[order_all['status'] == 1, 'status_name'] = '결제 완료'
order_all.loc[order_all['status'] == 2, 'status_name'] = '배달 완료'
order_all.loc[order_all['status'] == 9, 'status_name'] = '주문 취소'

order_all.loc[:, 'order_date'] = pd.to_datetime(
    order_all['order_accept_date']).dt.date

order_all.head()

C:\Users\creat\Documents\translate\book18\MLSys_100Knocks\MainChapter\chapter04\tbl_order_202104.csv:233262
C:\Users\creat\Documents\translate\book18\MLSys_100Knocks\MainChapter\chapter04\tbl_order_202105.csv:241142
C:\Users\creat\Documents\translate\book18\MLSys_100Knocks\MainChapter\chapter04\tbl_order_202106.csv:233302


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_date
0,34104383,11,C65806632,57,61573513,2021-04-01 11:00,2021-04-01 11:39,1,28270,1,자양로점,SL,서울,서울,takeout,결제 완료,2021-04-01
1,70652318,59,C09760173,37,54068709,2021-04-01 11:00,2021-04-01 11:34,0,28270,2,가마산로2점,SL,서울,서울,delivery,배달 완료,2021-04-01
2,71640388,195,C61227084,17,93678366,2021-04-01 11:00,2021-04-01 11:54,0,26470,9,화중로점,GB,경기,경기북부,delivery,주문 취소,2021-04-01
3,75673365,127,C64119972,17,5287952,2021-04-01 11:00,2021-04-01 11:17,0,23080,2,분포로점,BS,부산,부산,delivery,배달 완료,2021-04-01
4,9077529,174,C10231192,18,18248867,2021-04-01 11:00,2021-04-01 11:35,0,46920,2,하남산단점,GJ,광주,광주,delivery,배달 완료,2021-04-01


In [2]:
import openpyxl

wb = openpyxl.Workbook()
ws = wb['Sheet']
ws.cell(1,1).value = '파일 쓰기 테스트입니다.'
wb.save('test.xlsx')
wb.close()

In [3]:
wb = openpyxl.load_workbook('test.xlsx', read_only=True)
ws = wb['Sheet']
print(ws.cell(1, 1).value)
wb.close()

파일 쓰기 테스트입니다.


In [4]:
# 테스트 데이터 준비
store_id = 1
store_df = order_all.loc[order_all['store_id'] == store_id].copy()
store_name = store_df['store_name'].unique()[0]
store_sales_total = store_df.loc[
    store_df['status'].isin([1, 2])]['total_amount'].sum()
store_sales_takeout = store_df.loc[
    store_df['status'] == 1]['total_amount'].sum()
store_sales_delivery = store_df.loc[
    store_df['status'] == 2]['total_amount'].sum()
print(f'매출액 확인 {store_sales_total} = '
      f'{store_sales_takeout + store_sales_delivery}')
output_df = store_df[['order_accept_date', 'customer_id', 'total_amount',
                      'takeout_name', 'status_name']]
output_df.head()

매출액 확인 90101050 = 90101050


,order_accept_date,customer_id,total_amount,takeout_name,status_name
48,2021-04-01 11:03,C81203759,30500,takeout,결제 완료
435,2021-04-01 11:32,C93103282,26150,takeout,결제 완료
627,2021-04-01 11:50,C33623566,41440,delivery,주문 취소
642,2021-04-01 11:51,C64109769,33000,delivery,배달 완료
976,2021-04-01 12:21,C46929234,26030,delivery,주문 취소


In [5]:
from openpyxl.utils.dataframe import dataframe_to_rows

store_title = f'{store_id}_{store_name}'

wb = openpyxl.Workbook()
ws = wb.active
ws.title = store_title

ws.cell(1, 1).value = f'{store_title} 매출 데이터'

# OpenPyXL 유틸리티 dataframe_to_rows를 이용
rows = dataframe_to_rows(output_df, index=False, header=True)

# 테이블 삽입 위치
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value

filename = f'{store_title}.xlsx'
wb.save(filename)
wb.close()

### 테크닉 32: 엑셀 테이블을 정리해 출력하자

In [6]:
# 스타일 관련 임포트
from openpyxl.styles import PatternFill, Border, Side, Font

openpyxl.load_workbook(filename)
ws = wb[store_title]

side = Side(style='thin', color='008080')
border = Border(top=side, bottom=side, left=side, right=side)

# 데이터의 표 부분 테두리 설정
for row in ws:
    for cell in row:
        if ws[cell.coordinate].value:
            ws[cell.coordinate].border = border

In [7]:
ws.cell(1,1).font = Font(bold=True, color='008080')

cell = ws.cell(3, 2)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 접수 일시'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3, 3)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '고객 ID'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3, 4)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '수입 총액'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3, 5)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 타입'
cell.font = Font(bold=True, color='FFFFFF')

cell = ws.cell(3, 6)
cell.fill = PatternFill(patternType='solid', fgColor='008080')
cell.value = '주문 상태'
cell.font = Font(bold=True, color='FFFFFF')

ws.column_dimensions['A'].width = 20
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 12
ws.column_dimensions['D'].width = 12
ws.column_dimensions['E'].width = 12
ws.column_dimensions['F'].width = 12

# 파일에 저장
wb.save(filename)
wb.close()


### 테크닉 33: 매출 이외의 데이터도 출력하자

In [9]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


store_df.loc[:, 'order_accept_datetime'] = pd.to_datetime(
    store_df['order_accept_date'])
store_df.loc[:, 'delivered_datetime'] = pd.to_datetime(
    store_df['delivered_date'])
store_df.loc[:, 'delta'] = store_df[
    ['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)

delivery_time = store_df.groupby(['store_id'])['delta'].describe()
delivery_time

,count,mean,std,min,25%,50%,75%,max
store_id,,,,,,,,
1,3553.0,34.46327,14.502363,10.0,22.0,34.0,47.0,59.0


In [10]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(1, 7)
cell.value = f'배달 완료 소요 시간'
cell.font = Font(bold=True, color='008080')

rows = dataframe_to_rows(delivery_time, index=False, header=True)

# 표 삽입 위치
row_start = 3
col_start = 8

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        cell = ws.cell(row_no, col_no)
        cell.value = value
        cell.border = border
        if row_no == row_start:
            cell.fill = PatternFill(patternType='solid', fgColor='008080')
            cell.font = Font(bold=True, color='FFFFFF')
        
filename = f'{store_title}.xlsx'
wb.save(filename)
wb.close()

### 테크닉 34: 문제가 있는 위치를 빨간색으로 표시하자

In [11]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

rows = dataframe_to_rows(output_df, index=False, header=True)

# 표 삽입 위치
row_start = 3
col_start = 2


for row_no, row in enumerate(rows, row_start):
    if row_no == row_start:
        continue
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value
        if value == '주문 취소':
            ws.cell(row_no, col_no).font = Font(bold=True, color='FF0000')

filename = f'{store_title}.xlsx'
wb.save(filename)
wb.close()

### 테크닉 35: 엑셀의 셀 함수를 이용해 일단위로 집계하자

In [12]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7, 7)
cell.value = '집계'
cell.font = Font(bold=True, color='008080')

cell = ws.cell(8, 8)
cell.value = '데이터 총액'
cell.font = Font(bold=True, color='008080')

cell = ws.cell(8, 10)
cell.value = f'=SUM(D4:D{ws.max_row})'

cell = ws.cell(9, 8)
cell.value = '내 결제 완료 금액'
cell.font = Font(bold=True)

cell = ws.cell(9, 10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"<>주문 취소",D4:D{ws.max_row})'

cell = ws.cell(10, 8)
cell.value = '내 주문 취소 금액'
cell.font = Font(bold=True)

cell = ws.cell(10, 10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"=주문 취소",D4:D{ws.max_row})'

filename = f'{store_title}.xlsx'
wb.save(filename)
wb.close()

### 테크닉 36: 꺾은선 그래프로 출력하자

In [14]:
from openpyxl.chart import Reference, BarChart, PieChart, \
    LineChart, ScatterChart, Series

openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7, 7)
cell.value = f'매출 그래프'
cell.font = Font(bold=True, color='008080')

# 그래프용 참조 데이터를 지정한다. D열(구입 총액)의 4번째 행부터 20건을 지정
refy = Reference(ws, min_col=4, min_row=4, max_col=4, max_row=23)

# 그래프 시계열 생성
series = Series(refy, title='매출액')

# Chart
chart = LineChart()
chart.title = '꺾은선 그래프'
chart.x_axis.title = '건수'
chart.y_axis.title = '매출액'
chart.height = 10
chart.width = 20
chart.series.append(series)

# 생선한 Chart 객체를 시트의 지정한 위치에 추가
ws.add_chart(chart, 'H12')

filename = f'{store_title}.xlsx'
wb.save(filename)
wb.close()

### 테크닉 37: 보고서용 데이터를 준비하자

In [15]:
# 주문 취소율 순위 데이터 준비
cancel_df = pd.DataFrame()
cancel_cnt = order_all.loc[order_all['status'] == 9].groupby(
    ['store_id'])['store_id'].count()
order_cnt = order_all.loc[order_all['status'].isin([1, 2, 9])].groupby(
    ['store_id'])['store_id'].count()
cancel_rate = (cancel_cnt / order_cnt) * 100
cancel_df['cancel_rate'] = cancel_rate
cancel_df = pd.merge(cancel_df, m_store, on='store_id', how='left')
cancel_rank = cancel_df.sort_values('cancel_rate', 
                                    ascending=True).reset_index()


def check_store_cancel_rank(trg_id):
    tmp = cancel_rank.loc[cancel_rank['store_id'] == trg_id].index + 1
    return tmp[0]

In [16]:
def get_area_df(trg_id):
    # 해당 매장이 속한 지역별 데이터 집계와 매출 순위
    area_df = pd.DataFrame()
    area_df = order_all.loc[order_all['area_cd'] ==
                            store_df['area_cd'].unique()[0]]
    area_df = area_df.loc[area_df['status'].isin([1, 2])]
    return area_df


def get_area_rank_df(trg_id):
    area_df = get_area_df(trg_id)
    area_rank = area_df.groupby(['store_id'])['total_amount'].sum(). \
        sort_values(ascending=False)
    area_rank = pd.merge(area_rank, m_store, on='store_id', how='left')
    return area_rank


def check_store_sales_rank(trg_id):
    area_rank = get_area_rank_df(trg_id)
    tmp = area_rank.loc[area_rank['store_id'] == trg_id].index + 1
    return tmp[0]

In [17]:
def make_store_daily(trg_id):
    # 해당 매장의 일일 매출 데이터
    tmp_store_df = order_all.loc[(order_all['store_id'] == trg_id) &
                                 (order_all['status'].isin([1, 2]))]
    tmp = tmp_store_df[['order_accept_date', 'total_amount']].copy()
    tmp.loc[:, 'order_accept_date'] = pd.to_datetime(tmp['order_accept_date'])
    tmp.set_index('order_accept_date', inplace=True)
    tmp = tmp.resample('D').sum().reset_index()
    return tmp

In [18]:
def get_area_delivery(trg_id):
    # 해당 점포가 속한 지역별 데이터의 배달 완료까지의 시간 순위
    area_delivery = pd.DataFrame()
    area_df = get_area_df(trg_id)
    area_delivery = area_df.loc[area_df['status'] == 2].copy()

    area_delivery.loc[:, 'order_accept_datetime'] = pd.to_datetime(
        area_delivery['order_accept_date'])
    area_delivery.loc[:, 'delivered_datetime'] = pd.to_datetime(
        area_delivery['delivered_date'])
    area_delivery.loc[:, 'delta'] = area_delivery[[
        'order_accept_datetime', 'delivered_datetime']]. \
        apply(calc_delta, axis=1)
    return area_delivery


def get_area_delivery_rank_df(trg_id):
    area_delivery = get_area_delivery(trg_id)
    area_delivery_rank = area_delivery.groupby(['store_id'])['delta']. \
        mean().sort_values()
    area_delivery_rank = pd.merge(area_delivery_rank, m_store,
                                  on='store_id', how='left')
    return area_delivery_rank


def check_store_delivery_rank(trg_id):
    area_delivery_rank = get_area_delivery_rank_df(trg_id)
    tmp = area_delivery_rank.loc[
              area_delivery_rank['store_id'] == trg_id].index + 1
    return tmp[0]


### 테크닉 38: 데이터시트에 필요한 데이터를 출력하자

In [19]:
# 테스트용 파일 삭제
if os.path.exists('test.xlsx') : os.remove('test.xlsx')
if os.path.exists(filename): os.remove(filename)

In [20]:
def data_sheet_output(trg_wb, sheet_name, target_df, index_flg):
    ws = trg_wb.create_sheet(title=sheet_name)
    rows = dataframe_to_rows(target_df, index=index_flg, header=True)

    # 표 삽입 위치
    row_start = 1
    col_start = 1

    for row_no, row in enumerate(rows, row_start):
        for col_no, value in enumerate(row, col_start):
            ws.cell(row_no, col_no).value = value

    # 데이터시트는 표시하지 않음
    ws.sheet_state = 'hidden'

In [21]:
def make_data_sheet(trg_id, trg_st_df, target_folder):
    target_daily = make_store_daily(trg_id)
    store_name = trg_st_df['store_name'].unique()[0]

    # 새로운 파일을 만듬
    store_title = f'{trg_id}_{store_name}'

    wb = openpyxl.Workbook()

    # 주문 취소 순위
    data_sheet_output(wb, 'Data_CancelRank',
                      cancel_rank, False)
    # 지역 매출 순위
    data_sheet_output(wb, 'Data_AreaRank',
                      get_area_rank_df(trg_id), False)
    # 지역 배달 완료 시간 순위
    data_sheet_output(wb, 'Data_DeliveryRank',
                      get_area_delivery_rank_df(trg_id), False)
    # 해당 매장의 일단위 매출 데이터
    data_sheet_output(wb, 'Data_Target_Daily',
                      target_daily, False)

    filename = os.path.join(target_folder, f'{store_title}.xlsx')
    wb.save(filename)
    wb.close()

    return filename

In [22]:
filename_store = make_data_sheet(store_id, store_df, '')

### 테크닉 39: 요약 시트를 만들자

In [23]:
def make_summary_sheet(trg_id, storename, trgfile):
    target_cancel_rank = check_store_cancel_rank(trg_id)
    target_sales_rank = check_store_sales_rank(trg_id)
    target_delivery_rank = check_store_delivery_rank(trg_id)

    wb = openpyxl.load_workbook(trgfile)
    ws = wb.active
    ws.title = '요약 보고서'

    cell = ws.cell(1, 1)
    cell.value = f'{storename} 요약 보고서(4월~6월)'
    cell.font = Font(bold=True, color='008080', size=20)

    # 매출 순위 표시
    tmpWs = wb['Data_Target_Daily']
    cell = ws.cell(3, 2)
    cell.value = '매장 매출액'
    cell.font = Font(bold=True, color='008080', size=16)

    # 셀 병합
    ws.merge_cells('E3:F3')

    cell = ws.cell(3, 5)
    cell.value = f'=SUM({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.font = Font(bold=True, color='0080FF', size=16)
    cell.number_format = '#,##0'

    cell = ws.cell(4, 2)
    cell.value = '매장 매출 순위'
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(4, 5)
    cell.value = f'{len(m_store)}개 매장 중 {target_sales_rank}위'
    cell.font = Font(bold=True, color='0080FF', size=16)

    # 그래프용 참조 데이터 지정
    refy = Reference(tmpWs, min_col=2, min_row=2, max_col=2,
                     max_row=tmpWs.max_row)

    # 그래프 시계열 생성
    series = Series(refy, title='매출액')

    # Chart
    chart = LineChart()
    chart.title = '기간 매출액(일별)'
    chart.x_axis.title = '건수'
    chart.y_axis.title = '매출액'
    chart.height = 10
    chart.width = 15
    chart.series.append(series)

    # 생성한 Chart 객체를 시트의 지정한 위치에 추가
    ws.add_chart(chart, 'B6')

    # 지역 정보
    tmpWs = wb['Data_AreaRank']

    cell = ws.cell(4, 10)
    cell.value = '지역 매장 매출 정보'
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(5, 11)
    cell.value = '최고 금액'

    cell = ws.cell(5, 12)
    cell.value = f'=MAX({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = '#,##0'

    cell = ws.cell(6, 11)
    cell.value = '최저 금액'

    cell = ws.cell(6, 12)
    cell.value = f'=MIN({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = '#,##0'

    cell = ws.cell(7, 11)
    cell.value = '지역 평균'

    cell = ws.cell(7, 12)
    cell.value = f'=AVERAGE({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = '#,##0'

    # 주문 취소율 표시
    cell = ws.cell(11, 10)
    cell.value = '주문 취소 순위'
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(12, 11)
    cell.value = f'{len(m_store)}개 매장 중 {target_cancel_rank}위'
    cell.font = Font(bold=True, color='0080FF', size=16)

    tmpWs = wb['Data_CancelRank']

    cell = ws.cell(13, 11)
    cell.value = '지역 평균'

    cell = ws.cell(13, 12)
    cell.value = f'=AVERAGE({tmpWs.title}!C2:C{tmpWs.max_row})'
    cell.number_format = '0.00'

    # 배달 시간 순위 표시
    cell = ws.cell(15, 10)
    cell.value = '배달 완료 시간 순위'
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(16, 11)
    cell.value = f'{len(m_store)}개 매장 중 {target_delivery_rank}위'
    cell.font = Font(bold=True, color='0080FF', size=16)

    tmpWs = wb['Data_DeliveryRank']

    cell = ws.cell(17, 11)
    cell.value = '지역 평균'

    cell = ws.cell(17, 12)
    cell.value = f'=AVERAGE({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format = '0.00'

    wb.save(trgfile)
    wb.close()

In [24]:
make_summary_sheet(store_id, store_name, filename_store)

### 테크닉 40: 매장별 보고서를 엑셀로 출력하자

In [25]:
os.makedirs('output',exist_ok=True)

for store in m_store['store_id'].tolist():
    if store != 999:
        store_df = order_all.loc[order_all['store_id']==store]
        store_name = m_store.loc[m_store['store_id']==store]['store_name']
        print(store_name)
               
        tmp_file_name = make_data_sheet(store, store_df, 'output')
        make_summary_sheet(store, store_name.values[0], tmp_file_name)
        
print('출력을 완료했습니다.')

0    삼일대로점
Name: store_name, dtype: object
1    세종대로점
Name: store_name, dtype: object
2    무교로점
Name: store_name, dtype: object
3    덕수궁길점
Name: store_name, dtype: object
4    서소문로점
Name: store_name, dtype: object
5    청계청로점
Name: store_name, dtype: object
6    노해로점
Name: store_name, dtype: object
7    당산로점
Name: store_name, dtype: object
8    남부순환로점
Name: store_name, dtype: object
9    성내로점
Name: store_name, dtype: object
10    자양로점
Name: store_name, dtype: object
11    봉화산로점
Name: store_name, dtype: object
12    창경궁로점
Name: store_name, dtype: object
13    월드컵로점
Name: store_name, dtype: object
14    학동로점
Name: store_name, dtype: object
15    삼봉로점
Name: store_name, dtype: object
16    관악로점
Name: store_name, dtype: object
17    가마산로점
Name: store_name, dtype: object
18    목공동로점
Name: store_name, dtype: object
19    장승배기로점
Name: store_name, dtype: object
20    고산자로점
Name: store_name, dtype: object
21    녹사평대로점
Name: store_name, dtype: object
22    시흥대로점
Name: store_name, dtype: object
23 

185    효원로점
Name: store_name, dtype: object
186    영통구점
Name: store_name, dtype: object
187    처인구점
Name: store_name, dtype: object
188    성남대로점
Name: store_name, dtype: object
189    기흥점
Name: store_name, dtype: object
190    사우중로점
Name: store_name, dtype: object
191    부흥로점
Name: store_name, dtype: object
192    방죽로점
Name: store_name, dtype: object
193    일산동구점
Name: store_name, dtype: object
194    화중로점
Name: store_name, dtype: object
195    석봉로점
Name: store_name, dtype: object
출력을 완료했습니다.
